In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload 'cybersecurity_large_synthesized_data.csv'


Saving cybercrime_mitigation_dataset.csv to cybercrime_mitigation_dataset.csv


In [ ]:
!pip install transformers datasets pandas torch


  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import torch
import os

# Load and clean dataset
df = pd.read_csv("cybercrime_mitigation_dataset.csv")
df.columns = df.columns.str.strip()  # Remove whitespace

# Fill NAs with blank
df = df.fillna("")

# Build prompt-completion format
def make_prompt_completion(row):
    if row['Identification']:
        prompt = f"Scenario: {row['Identification']}\nCybercrime:"
        completion = f" {row['Cybercrime']}\nMitigation Strategies: {row['Mitigation Strategy 1']}, {row['Mitigation Strategy 2']}, {row['Mitigation Strategy 3']}"
    else:
        prompt = f"Cybercrime: {row['Cybercrime']}\nIdentification:"
        completion = f" {row['Identification']}\nMitigation Strategies: {row['Mitigation Strategy 1']}, {row['Mitigation Strategy 2']}, {row['Mitigation Strategy 3']}"
    return prompt + completion

df['text'] = df.apply(make_prompt_completion, axis=1)

# Save to text file for training
with open("cybercrime_dataset.txt", "w", encoding="utf-8") as f:
    for line in df['text']:
        f.write(line + "\n")

# Load tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT2 doesn't have padding token

model = GPT2LMHeadModel.from_pretrained(model_name)

# Create dataset
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size)
    return dataset

train_dataset = load_dataset("cybercrime_dataset.txt", tokenizer)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-cybercrime",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=50,
    warmup_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

# Fine-tune the model
trainer.train()

# Save final model
trainer.save_model("./gpt2-cybercrime")
tokenizer.save_pretrained("./gpt2-cybercrime")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kamaleshwaran-s2022 (eshaan-rithesh2023-vit-chennai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,1.792500
100,0.482800
150,0.256600


('./gpt2-cybercrime/tokenizer_config.json',
 './gpt2-cybercrime/special_tokens_map.json',
 './gpt2-cybercrime/vocab.json',
 './gpt2-cybercrime/merges.txt',
 './gpt2-cybercrime/added_tokens.json')

In [ ]:
from transformers import pipeline

model_path = "./gpt2-cybercrime"
generator = pipeline("text-generation", model=model_path, tokenizer=model_path, device=-1)  # force CPU

prompt = "Scenario: redirected to a suspicious website unexpectedly\nCybercrime:"
output = generator(
    prompt,
    max_new_tokens=50,  # limit how much to generate
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    truncation=True
)

print(output[0]["generated_text"])


Device set to use cpu


Scenario: redirected to a suspicious website unexpectedly
Cybercrime: DNS Spoofing
Mitigation Strategies: Use DNSSEC, Monitor DNS traffic, Use secure DNS providers
Scenario: Malware installs just by visiting a compromised website.
Cybercrime: Drive-by Download
Mitigation Strategies:


In [ ]:
from transformers import pipeline

# Load fine-tuned model and tokenizer
model_path = "./gpt2-cybercrime"
generator = pipeline("text-generation", model=model_path, tokenizer=model_path, device=-1)

# Define test prompts
prompts = [
    "Scenario: Victim receives an email from a trusted contact, but the message contains unusual language and a suspicious attachment.\nCybercrime:",
    "Scenario: A user notices that the date typed on a login page is being recorded and transmitted to an attacker.\nCybercrime:"
]

# Generate and print predictions
for i, prompt in enumerate(prompts, 1):
    output = generator(
        prompt,
        max_new_tokens=50,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        truncation=True
    )
    print(f"\n🔹 Example {i}")
    print(output[0]["generated_text"])


Device set to use cpu



🔹 Example 1
Scenario: Victim receives an email from a trusted contact, but the message contains unusual language and a suspicious attachment.
Cybercrime: Phishing
Mitigation Strategies: Implement email filtering, Educate users on phishing, Use multi-factor authentication
Scenario: Multiple failed login attempts from unknown IPs.
Cybercrime: Credential Stuffing
Mitigation Strategies

🔹 Example 2
Scenario: A user notices that the date typed on a login page is being recorded and transmitted to an attacker.
Cybercrime: Session Hijacking
Mitigation Strategies: Use secure cookies, Implement session timeouts, Use Content Security Policy
Scenario: Unexpected scripts executing in the browser.
Cybercrime: Cross-Site Scripting (XSS)
Mit


In [ ]:
from transformers import pipeline

model_path = "./gpt2-cybercrime"
generator = pipeline("text-generation", model=model_path, tokenizer=model_path, device=-1)  # force CPU

prompt = "Credential stuffing"
output = generator(
    prompt,
    max_new_tokens=50,  # limit how much to generate
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    truncation=True
)

print(output[0]["generated_text"])


Device set to use cpu


Credential stuffing, Exploitation of unknown software vulnerabilities, Zero-Day Exploit
Scenario: Network or website becomes unresponsive due to traffic overload.
Cybercrime: DDoS Attack
Mitigation Strategies: Use rate limiting, Deploy web application firewall
